In [ ]:
### CODE BLOCK 1
%pip install -q langchain langchain-community langchain-nvidia-ai-endpoints gradio rich
%pip install -q arxiv pymupdf faiss-cpu

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
### CODE BLOCK 2
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set NVIDIA API key from environment variable
# If not found in environment, this will be None
api_key = os.environ.get("NVIDIA_API_KEY")

# Print a message based on whether the key was found
if api_key:
    print("NVIDIA API key loaded successfully")
else:
    raise ValueError("NVIDIA API key not found in .env file. Please set NVIDIA_API_KEY in your environment variables.")


NVIDIA API key loaded successfully


In [ ]:
### CODE BLOCK 3
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# Get all available models
all_models = ChatNVIDIA.get_available_models()

# Display only the first 20 models
print(f"First 20 of {len(all_models)} available models:")
for i, model in enumerate(all_models[:20]):
    print(f"{i+1}. {model}")

First 20 of 88 available models:
1. id='institute-of-science-tokyo/llama-3.1-swallow-70b-instruct-v0.1' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=False supports_structured_output=True base_model=None
2. id='nvidia/llama-3.1-nemotron-51b-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=False supports_structured_output=False base_model=None
3. id='qwen/qwen2.5-coder-32b-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=False supports_structured_output=False base_model=None
4. id='google/gemma-7b' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=['ai-gemma-7b', 'playground_gemma_7b', 'gemma_7b'] supports_tools=False supports_structured_output=False base_model=None
5. id='yentinglin/llama-3-taiwan-70b-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=False supports_structured_output=False base_model=None
6. id='microsoft/k

In [ ]:
### CODE BLOCK 4
###################################### Document Summarization  ##################################

from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader, ArxivLoader

from pydantic import BaseModel, Field
from typing import List
from IPython.display import clear_output
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

# Define the Pydantic model for document summarization
class DocumentSummaryBase(BaseModel):
    """Base class for document summarization results."""
    running_summary: str = Field(default="", description="The running summary of the document. Update, don't override.")
    main_ideas: List[str] = Field(default_factory=list, description="Maximum 3 important points from the document.")
    loose_ends: List[str] = Field(default_factory=list, description="Maximum 3 open questions or areas needing clarification.")

# Set up console formatting with NVIDIA brand color
console = Console()
nvidia_green = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=nvidia_green)

# Document Loading 
# For local files (commented out as an option)
# loader = UnstructuredFileLoader("your_document.pdf")
# docs = loader.load()

# Load document from Arxiv (GraphRAG paper)
loader = ArxivLoader(query="2404.16130")  # GraphRAG paper
docs = loader.load()

# Configure text splitter for document chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, 
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " ", ""],
)

# Optional preprocessing (commented out)
# def preprocess_text(text):
#     # Remove special characters, normalize whitespace, etc.
#     return text.replace("...", ".")

# Split documents into manageable chunks
docs_split = text_splitter.split_documents(docs)
print(f"Document split into {len(docs_split)} chunks")

# Create summarization prompt
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are a technical document summarizer tasked with generating a running summary of a research document.
    You are given chunks of text from a document and an existing knowledge base containing:
    1. A running summary of the document so far
    2. Main ideas (max 3)
    3. Loose ends or questions (max 3)
    
    Your goal is to update this knowledge base with new information from the current document chunk.
    
    IMPORTANT INSTRUCTIONS:
    - DO NOT lose information when updating the knowledge base
    - Integrate new information with existing running_summary
    - Update main_ideas and loose_ends as needed, but keep them limited to 3 items each
    - Follow the format instructions exactly
    - If nothing new is present in the chunk, return the existing knowledge base unchanged
    
    {format_instructions}
    """),
    ("user", "{input}")
])




Document split into 83 chunks


In [ ]:
### CODE BLOCK 5

def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace(r"\]", "]")
            .replace(r"\[", "[")
        )
        return string
    return instruct_merge | prompt | llm | preparse | parser


In [ ]:
### CODE BLOCK 6

latest_summary = ""

def RSummarizer(knowledge, llm, prompt, verbose=False):
    def summarize_docs(docs):
        parse_chain = RunnableAssign({'info_base' : RExtract(knowledge.__class__, llm, prompt)})
        state = {'info_base' : knowledge}

        global latest_summary  ## If your loop crashes, you can check out the latest_summary

        for i, doc in enumerate(docs):
            state['input'] = doc.page_content
            state = parse_chain.invoke(state)

            assert 'info_base' in state
            if verbose:
                print(f"Considered {i+1} documents")
                pprint(state['info_base'])
                latest_summary = state['info_base']
                clear_output(wait=True)

        return state['info_base']
    return RunnableLambda(summarize_docs)

instruct_model = ChatNVIDIA(model="mistralai/mistral-7b-instruct-v0.3").bind(max_tokens=4096)
instruct_llm = instruct_model | StrOutputParser()

## Take the first 10 document chunks and accumulate a DocumentSummaryBase
summarizer = RSummarizer(DocumentSummaryBase(), instruct_llm, summary_prompt, verbose=True)
summary = summarizer.invoke(docs_split[:10])

{   "description": "Base class for document summarization results.",   "properties": {     "running_summary": "The document discusses a new method called GraphRAG that creates summaries for nested modular communities in a graph. It also mentions the necessity of adaptive benchmarking for evaluating the performance of this method.",     "main_ideas": ["GraphRAG for creating summaries for nested modular communities", "Adaptive benchmarking for evaluating GraphRAG"],     "loose_ends": ["Details about the LLM used in GraphRAG", "More information about the proposed method for generating questions for evaluation", "Information about other benchmark datasets and their focus on vector RAG performance"]   } }
Considered 10 documents


DocumentSummaryBase(running_summary='', main_ideas=[], loose_ends=[])

In [ ]:
### CODE BLOCK 7
########################################### Retrieval-Augmented Generation (RAG) ###########################################

# Standard library imports
import json
from operator import itemgetter

# UI components
import gradio as gr

# NVIDIA specific components
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# Vector storage components
from langchain_community.vectorstores import FAISS
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

# Document processing components
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ArxivLoader, UnstructuredPDFLoader

# Document transformation
from langchain.document_transformers import LongContextReorder

# LangChain components
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign

# Utility imports
from functools import partial

# Rich console formatting
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

# Set up console formatting with NVIDIA brand color
console = Console()
nvidia_green = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=nvidia_green)

# Configure embedding model
# NVIDIAEmbeddings.get_available_models()  # Get list of available embedding models
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# Set up LLM 
# ChatNVIDIA.get_available_models()  # Get list of available LLM models
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

# Utility function for pretty printing in chains
def RPrint(preface=None):
    """
    Creates a RunnableLambda that prints input with NVIDIA styling and returns it unchanged.
    Useful for debugging chains by showing intermediate values.
    
    Args:
        preface: Optional text to print before the input
        
    Returns:
        RunnableLambda: A function that prints then returns its input
    """
    def print_and_return(x):
        msg = f"{preface}: {x}" if preface else x
        pprint(msg)
        return x
    
    return RunnableLambda(print_and_return)

# Function to format document chunks as a string
def docs2str(docs, title=None):
    """
    Formats a list of document chunks into a readable string format.
    
    Args:
        docs: List of document objects with page_content and metadata
        title: Optional title to include in the output
        
    Returns:
        str: Formatted string representation of the documents
    """
    if not docs:
        return "(No relevant documents found)"
    
    result = f"--- {title} ---\n\n" if title else ""
    
    for i, doc in enumerate(docs):
        source_info = ""
        if hasattr(doc, 'metadata') and doc.metadata:
            if 'title' in doc.metadata:
                source_info = f"[Quote from {doc.metadata['title']}]"
            elif 'source' in doc.metadata:
                source_info = f"[Quote from {doc.metadata['source']}]"
        
        result += f"{source_info}\n{doc.page_content}\n\n"
    
    return result

# Create document transformer for reordering documents for better context
long_reorder = RunnableLambda(lambda docs: LongContextReorder().transform_documents(docs))

# Configure text splitter for document chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "]
)

# Load academic papers using ArxivLoader
print("Loading Documents")
documents = []
arxiv_papers = [
    ("1706.03762", "Attention Is All You Need"),  # Transformer architecture
    ("1810.04805", "BERT"),  # Bidirectional Encoder Representations from Transformers
    ("2005.11401", "RAG"),  # Retrieval-Augmented Generation
    ("2205.00445", "MRKL"),  # Modular Reasoning, Knowledge and Language
    ("2310.06825", "Mistral"),  # Mistral LLM
    ("2306.05685", "LLM-as-a-Judge"),  # Using LLMs for evaluations
    ("2210.03629", "ReAct"),  # Reasoning and Acting in LLMs
    ("2112.10752", "Latent Stable Diffusion"),  # Diffusion Models
    ("2103.00020", "CLIP")   # Contrastive Language-Image Pre-training
]

for paper_id, description in arxiv_papers:
    try:
        loader = ArxivLoader(query=paper_id)
        doc = loader.load()
        if doc:
            # Add description to metadata
            for d in doc:
                if not hasattr(d, 'metadata'):
                    d.metadata = {}
                d.metadata['description'] = description
            documents.extend(doc)
            print(f"Loaded: {paper_id} - {description}")
    except Exception as e:
        print(f"Failed to load {paper_id}: {e}")

print(f"Loaded {len(documents)} documents")

# Process the loaded papers
# Convert to JSON and truncate at References section if present
processed_docs = []
for doc in documents:
    content = doc.page_content
    
    # Truncate content at References section to focus on the main content
    ref_indices = [content.find("\nReferences\n"), content.find("\nREFERENCES\n")]
    ref_indices = [idx for idx in ref_indices if idx != -1]
    
    if ref_indices:
        # Truncate at the first occurrence of References
        content = content[:min(ref_indices)]
    
    doc.page_content = content
    processed_docs.append(doc)

# Split documents into chunks
print("Chunking Documents")
docs_chunks = []
for doc in processed_docs:
    # Split document into chunks
    chunks = text_splitter.split_documents([doc])
    
    # Filter out very short chunks (likely section headers or incomplete sentences)
    filtered_chunks = [chunk for chunk in chunks if len(chunk.page_content) >= 200]
    
    docs_chunks.append(filtered_chunks)

# Create metadata and summary information
doc_string = "Available Documents:\n"
doc_metadata = []

# Build document summary string and metadata
for i, doc_chunk in enumerate(docs_chunks):
    if not doc_chunk:  # Skip empty document chunks
        continue
    
    # Extract document metadata
    metadata = doc_chunk[0].metadata
    title = metadata.get('title', f"Document {i+1}")
    description = metadata.get('description', '')
    
    # Add to document string
    doc_string += f"• {title} - {description}\n"
    
    # Add to metadata
    doc_metadata.append({
        'index': i,
        'title': title,
        'description': description,
        'chunk_count': len(doc_chunk)
    })

# Create extra chunks with document metadata
extra_chunks = [
    doc_string,
    json.dumps(doc_metadata, indent=2)
]

# Print summary information
pprint(doc_string)

# Print detailed document information
for i, chunks in enumerate(docs_chunks):
    if not chunks:  # Skip empty document chunks
        continue
        
    metadata = chunks[0].metadata
    print(f"Document {i+1}:")
    print(f"  Chunks: {len(chunks)}")
    pprint(f"  Metadata: {metadata}")
    print()  # Empty line for better readability

C:\Users\dbenn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading Documents
Loaded: 1706.03762 - Attention Is All You Need
Loaded: 1810.04805 - BERT
Loaded: 2005.11401 - RAG
Loaded: 2205.00445 - MRKL
Loaded: 2310.06825 - Mistral
Loaded: 2306.05685 - LLM-as-a-Judge
Loaded: 2210.03629 - ReAct
Loaded: 2112.10752 - Latent Stable Diffusion
Loaded: 2103.00020 - CLIP
Loaded 9 documents
Chunking Documents


Available Documents:
• Document 1 - Attention Is All You Need
• Document 2 - BERT
• Document 3 - RAG
• Document 4 - MRKL
• Document 5 - Mistral
• Document 6 - LLM-as-a-Judge
• Document 7 - ReAct
• Document 8 - Latent Stable Diffusion
• Document 9 - CLIP

Document 1:
  Chunks: 34


  Metadata: {'Published': '2023-08-02', 'Title': 'Attention Is All You Need', 'Authors': 'Ashish Vaswani, Noam 
Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin', 'Summary': 
'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks in an 
encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder through an 
attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on attention 
mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks 
show these models to be\nsuperior in quality while being more parallelizable and requiring significantly\nless time
to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation task, improving over the 
existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 English-to-French\ntranslation task, 
our model establishes a new single-model state-of-the-art\nBLEU score of 41.8 after training for 3.5 days on eight 
GPUs, a small fraction\nof the training costs of the best models from the literature. We show that the\nTransformer
generalizes well to other tasks by applying it successfully to\nEnglish constituency parsing both with large and 
limited training data.', 'description': 'Attention Is All You Need'}

...

  Metadata: {'Published': '2023-03-10', 'Title': 'ReAct: Synergizing Reasoning and Acting in Language Models', 
'Authors': 'Shunyu Yao, Jeffrey Zhao, Dian Yu, Nan Du, Izhak Shafran, Karthik Narasimhan, Yuan Cao', 'Summary': 
'While large language models (LLMs) have demonstrated impressive capabilities\nacross tasks in language 
understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought prompting) and
acting (e.g.\naction plan generation) have primarily been studied as separate topics. In this\npaper, we explore 
the use of LLMs to generate both reasoning traces and\ntask-specific actions in an interleaved manner, allowing for
greater synergy\nbetween the two: reasoning traces help the model induce, track, and update\naction plans as well 
as handle exceptions, while actions allow it to interface\nwith external sources, such as knowledge bases or 
environments, to gather\nadditional information. We apply our approach, named ReAct, to a diverse set of\nlanguage 
and decision making tasks and demonstrate its effectiveness over\nstate-of-the-art baselines, as well as improved 
human interpretability and\ntrustworthiness over methods without reasoning or acting components.\nConcretely, on 
question answering (HotpotQA) and fact verification (Fever),\nReAct overcomes issues of hallucination and error 
propagation prevalent in\nchain-of-thought reasoning by interacting with a simple Wikipedia API, and\ngenerates 
human-like task-solving trajectories that are more interpretable than\nbaselines without reasoning traces. On two 
interactive decision making\nbenchmarks (ALFWorld and WebShop), ReAct outperforms imitation and\nreinforcement 
learning methods by an absolute success rate of 34% and 10%\nrespectively, while being prompted with only one or 
two in-context examples.\nProject site with code: https://react-lm.github.io', 'description': 'ReAct'}


Document 8:
  Chunks: 48


  Metadata: {'Published': '2022-04-13', 'Title': 'High-Resolution Image Synthesis with Latent Diffusion Models', 
'Authors': 'Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer', 'Summary': 'By 
decomposing the image formation process into a sequential application of\ndenoising autoencoders, diffusion models 
(DMs) achieve state-of-the-art\nsynthesis results on image data and beyond. Additionally, their formulation\nallows
for a guiding mechanism to control the image generation process without\nretraining. However, since these models 
typically operate directly in pixel\nspace, optimization of powerful DMs often consumes hundreds of GPU days 
and\ninference is expensive due to sequential evaluations. To enable DM training on\nlimited computational 
resources while retaining their quality and flexibility,\nwe apply them in the latent space of powerful pretrained 
autoencoders. In\ncontrast to previous work, training diffusion models on such a representation\nallows for the 
first time to reach a near-optimal point between complexity\nreduction and detail preservation, greatly boosting 
visual fidelity. By\nintroducing cross-attention layers into the model architecture, we turn\ndiffusion models into
powerful and flexible generators for general conditioning\ninputs such as text or bounding boxes and 
high-resolution synthesis becomes\npossible in a convolutional manner. Our latent diffusion models (LDMs) 
achieve\na new state of the art for image inpainting and highly competitive performance\non various tasks, 
including unconditional image generation, semantic scene\nsynthesis, and super-resolution, while significantly 
reducing computational\nrequirements compared to pixel-based DMs. Code is available 
at\nhttps://github.com/CompVis/latent-diffusion .', 'description': 'Latent Stable Diffusion'}


Document 9:
  Chunks: 143


  Metadata: {'Published': '2021-02-26', 'Title': 'Learning Transferable Visual Models From Natural Language 
Supervision', 'Authors': 'Alec Radford, Jong Wook Kim, Chris Hallacy, Aditya Ramesh, Gabriel Goh, Sandhini Agarwal,
Girish Sastry, Amanda Askell, Pamela Mishkin, Jack Clark, Gretchen Krueger, Ilya Sutskever', 'Summary': 
'State-of-the-art computer vision systems are trained to predict a fixed set\nof predetermined object categories. 
This restricted form of supervision limits\ntheir generality and usability since additional labeled data is needed 
to\nspecify any other visual concept. Learning directly from raw text about images\nis a promising alternative 
which leverages a much broader source of\nsupervision. We demonstrate that the simple pre-training task of 
predicting\nwhich caption goes with which image is an efficient and scalable way to learn\nSOTA image 
representations from scratch on a dataset of 400 million (image,\ntext) pairs collected from the internet. After 
pre-training, natural language\nis used to reference learned visual concepts (or describe new ones) 
enabling\nzero-shot transfer of the model to downstream tasks. We study the performance\nof this approach by 
benchmarking on over 30 different existing computer vision\ndatasets, spanning tasks such as OCR, action 
recognition in videos,\ngeo-localization, and many types of fine-grained object classification. The\nmodel 
transfers non-trivially to most tasks and is often competitive with a\nfully supervised baseline without the need 
for any dataset specific training.\nFor instance, we match the accuracy of the original ResNet-50 on 
ImageNet\nzero-shot without needing to use any of the 1.28 million training examples it\nwas trained on. We release
our code and pre-trained model weights at\nhttps://github.com/OpenAI/CLIP.', 'description': 'CLIP'}

In [22]:
%%time
### CODE BLOCK 8

# Constructing Your Document Vector Stores

def default_FAISS(embedder):
    """
    Create an empty FAISS vectorstore with proper configuration.
    
    Args:
        embedder: The embedding model to use for determining vector dimensions
        
    Returns:
        An empty FAISS vectorstore configured with IndexFlatL2 and InMemoryDocstore
    """
    # Get the embedding dimension from a sample embedding
    sample_embedding = embedder.embed_query("Sample text to determine dimensions")
    dimension = len(sample_embedding)
    
    # Create an empty FAISS index with L2 distance metric
    index = IndexFlatL2(dimension)
    
    # Create an empty docstore and index mapping
    docstore = InMemoryDocstore({})
    index_to_docstore_id = {}
    
    # Return the configured FAISS instance
    return FAISS(
        embedding_function=embedder,
        index=index,
        docstore=docstore,
        index_to_docstore_id=index_to_docstore_id,
        normalize_L2=False  # Don't normalize vectors as per requirements
    )

def aggregate_vstores(vecstores):
    """
    Efficiently merge multiple vector stores into a single vectorstore.
    
    Args:
        vecstores: List of FAISS vectorstores to merge
        
    Returns:
        A unified FAISS vectorstore containing all documents from input stores
    """
    # Create an empty FAISS instance to serve as the aggregation target
    # Using the embedding function from the first vectorstore
    merged_store = default_FAISS(vecstores[0].embedding_function)
    
    # Track the total number of vectors for progress reporting
    total_docs = 0
    
    # Iterate through each vectorstore and merge its contents
    for vs in vecstores:
        # Access the internal components of each FAISS instance
        docstore_dict = vs.docstore._dict
        
        # Collect texts, metadatas, and ids for batch processing
        texts = []
        metadatas = []
        ids = []
        
        # Process each document from the source vectorstore
        for doc_id, doc in docstore_dict.items():
            texts.append(doc.page_content)
            metadatas.append(doc.metadata)
            ids.append(doc_id)
            total_docs += 1
        
        # Batch add all documents from this vectorstore
        if texts:
            merged_store.add_texts(texts, metadatas, ids)
    
    print(f"Merged {len(vecstores)} vectorstores with {total_docs} total vectors")
    return merged_store

# Create vector store from metadata/summary chunks
print("Creating vectorstore for metadata and summaries...")
metadata_vectorstore = FAISS.from_texts(
    extra_chunks,  # The summary and metadata text chunks
    embedder,      # NVIDIA embeddings model
    metadatas=[{"source": "metadata"} for _ in extra_chunks]  # Tag these as metadata
)

# Flatten the list of document chunks before creating the vector store
# This fixes the "list object has no attribute page_content" error
print("Flattening document chunks...")
flattened_docs_chunks = []
for doc_chunk_list in docs_chunks:
    flattened_docs_chunks.extend(doc_chunk_list)

print(f"Total flattened document chunks: {len(flattened_docs_chunks)}")

# Create vector store from document chunks
print("Creating vectorstore for document chunks...")
docs_vectorstore = FAISS.from_documents(
    flattened_docs_chunks,  # Now a flat list of document chunks
    embedder               # NVIDIA embeddings model
)

# Combine the vectorstores
vecstores = [metadata_vectorstore, docs_vectorstore]

# Aggregate into a final docstore
print("Aggregating vectorstores...")
docstore = aggregate_vstores(vecstores)

# Print confirmation of total document count
total_chunks = len(docstore.docstore._dict)  # Use _dict instead of dict
print(f"✅ Vector store construction complete with {total_chunks} total document chunks available for retrieval")

# Optional: Memory optimization hint
print("Note: Large vectorstores have been merged efficiently to minimize memory usage")

Creating vectorstore for metadata and summaries...
Flattening document chunks...
Total flattened document chunks: 457
Creating vectorstore for document chunks...
Aggregating vectorstores...
Merged 2 vectorstores with 459 total vectors
✅ Vector store construction complete with 459 total document chunks available for retrieval
Note: Large vectorstores have been merged efficiently to minimize memory usage
CPU times: total: 953 ms
Wall time: 58.5 s


In [27]:
### CODE BLOCK 9 - RAG Chat Interface

# Set up models
# ChatNVIDIA.get_available_models()  # Uncomment to see available models
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
# Alternative model option: instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

# Set up conversation memory store
print("Creating conversation memory store...")
convstore = default_FAISS(embedder)

def save_memory_and_get_output(interaction_dict, vector_store):
    """
    Saves both the user question and agent response to memory as embedded texts.
    
    Args:
        interaction_dict: Dictionary containing 'input' (user message) and 'output' (agent response)
        vector_store: FAISS vector store for storing conversation history
        
    Returns:
        str: The agent's response (output value from the dictionary)
    """
    from datetime import datetime  # Added missing import
    
    user_text = f"User previously responded with {interaction_dict['input']}"
    agent_text = f"Agent previously responded with {interaction_dict['output']}"
    
    # Add both texts to the vector store with appropriate metadata
    vector_store.add_texts(
        [user_text, agent_text],
        metadatas=[
            {"role": "user", "timestamp": str(datetime.now())},
            {"role": "agent", "timestamp": str(datetime.now())}
        ]
    )
    
    return interaction_dict["output"]

# Create welcome message
initial_msg = f"""
# Welcome to Huang's Hype Factory Document Assistant!

I'm here to help you with information about the following documents:

{doc_string}

How can I help you understand these papers today?
"""

# Create chat prompt template
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are a knowledgeable research assistant who helps users understand technical AI papers. 
    
    The user's question is: {input}
    
    CONVERSATION HISTORY:
    {history}
    
    DOCUMENT CONTEXT:
    {context}
    
    INSTRUCTIONS:
    1. Only answer using information from the provided context and your understanding of AI concepts
    2. If you don't know or the answer isn't in the context, say so honestly
    3. Keep responses conversational but technically precise
    4. Do not make up information that isn't in the papers
    5. Include relevant quotes where helpful, citing the paper
    """),
    ("user", "{input}")
])

# Create streaming response chain
stream_chain = (
    chat_prompt 
    | RPrint("Prompt Sent to LLM") 
    | instruct_llm 
    | StrOutputParser()
)

# Fix: Create retrieval chain as a proper RunnableLambda
def create_context(message):
    """Function that creates context for the chat prompt."""
    # First create the base input
    context = {"input": message}
    
    # Add conversation history
    history_docs = convstore.similarity_search(message, k=5)
    history_docs = long_reorder.invoke(history_docs)
    history_str = docs2str(history_docs, title="Conversation History")
    context["history"] = history_str
    
    # Add document context
    doc_docs = docstore.similarity_search(message, k=8)
    doc_docs = long_reorder.invoke(doc_docs)
    doc_str = docs2str(doc_docs, title="Document Context")
    context["context"] = doc_str
    
    # Debug print
    pprint(f"Retrieved Context for: {message}")
    
    return context

retrieval_chain = RunnableLambda(create_context)

# Chat generator function
def chat_gen(message, history=[], return_buffer=True):
    """
    Generate a streaming chat response using RAG.
    
    Args:
        message: User's input message
        history: Chat history (not actively used as we have vector store)
        return_buffer: If True, return accumulated buffer; if False, return individual tokens
        
    Returns:
        Generator yielding either the growing buffer or individual tokens
    """
    buffer = ""
    
    # Perform retrieval to get context
    context = retrieval_chain.invoke(message)
    
    # Stream responses from the LLM
    for token in stream_chain.stream(context):
        buffer += token
        if return_buffer:
            yield buffer
        else:
            yield token
    
    # Save the conversation to memory
    save_memory_and_get_output({
        "input": message,
        "output": buffer
    }, convstore)

# Test the implementation
test_question = "Tell me about RAG!"
print("\nTesting chat implementation with query:", test_question)
print("\nResponse:")

for token in chat_gen(test_question, return_buffer=False):
    print(token, end='')

Creating conversation memory store...

Testing chat implementation with query: Tell me about RAG!

Response:


Retrieved Context for: Tell me about RAG!

Prompt Sent to LLM: messages=[SystemMessage(content="\n    You are a knowledgeable research assistant who helps 
users understand technical AI papers. \n    \n    The user's question is: Tell me about RAG!\n    \n    
CONVERSATION HISTORY:\n    (No relevant documents found)\n    \n    DOCUMENT CONTEXT:\n    --- Document Context 
---\n\n\nRAG-Token\nThe RAG-Token model can be seen as a standard, autoregressive seq2seq genera-\ntor with 
transition probability: p′\nθ(yi|x, y1:i−1) = P\nz∈top-k(p(·|x)) pη(zi|x)pθ(yi|x, zi, y1:i−1) To\ndecode, we can 
plug p′\nθ(yi|x, y1:i−1) into a standard beam decoder.\nRAG-Sequence\nFor RAG-Sequence, the likelihood p(y|x) does 
not break into a conventional per-\ntoken likelihood, hence we cannot solve it with a single beam search. Instead, 
we run beam search for\neach document z, scoring each hypothesis using pθ(yi|x, z, y1:i−1). This yields a set of 
hypotheses\nY , some of which may not have appeared in the beams of all documents. To estimate the probability\nof 
an hypothesis y we run an additional forward pass for each document z for which y does not\nappear in the beam, 
multiply generator probability with pη(z|x) and then sum the probabilities across\nbeams for the marginals. We 
refer to this decoding procedure as “Thorough Decoding.” For longer\n\n\nWe build RAG models where the parametric 
memory is a pre-trained seq2seq transformer, and the\nnon-parametric memory is a dense vector index of Wikipedia, 
accessed with a pre-trained neural\nretriever. We combine these components in a probabilistic model trained 
end-to-end (Fig. 1). The\nretriever (Dense Passage Retriever [26], henceforth DPR) provides latent documents 
conditioned on\nthe input, and the seq2seq model (BART [32]) then conditions on these latent documents together 
with\nthe input to generate the output. We marginalize the latent documents with a top-K approximation,\neither on 
a per-output basis (assuming the same document is responsible for all tokens) or a per-token\nbasis (where 
different documents are responsible for different tokens). Like T5 [51] or BART, RAG\ncan be ﬁne-tuned on any 
seq2seq task, whereby both the generator and retriever are jointly learned.\nThere has been extensive previous work
proposing architectures to enrich systems with non-parametric\n\n\nmemory by editing the document index. This 
approach has also been used in knowledge-intensive\ndialog, where generators have been conditioned on retrieved 
text directly, albeit obtained via TF-IDF\nrather than end-to-end learnt retrieval [9].\nRetrieve-and-Edit 
approaches\nOur method shares some similarities with retrieve-and-edit style\napproaches, where a similar training 
input-output pair is retrieved for a given input, and then edited\nto provide a ﬁnal output. These approaches have 
proved successful in a number of domains including\nMachine Translation [18, 22] and Semantic Parsing [21]. Our 
approach does have several differences,\nincluding less of emphasis on lightly editing a retrieved item, but on 
aggregating content from several\npieces of retrieved content, as well as learning latent retrieval, and retrieving
evidence documents\nrather than related training pairs. This said, RAG techniques may work well in these settings, 
and\ncould represent promising future work.\n6\nDiscussion\n\n\nextractive tasks, we ﬁnd that unconstrained 
generation outperforms previous extractive approaches.\nFor knowledge-intensive generation, we experiment with 
MS-MARCO [1] and Jeopardy question\ngeneration, and we ﬁnd that our models generate responses that are more 
factual, speciﬁc, and\ndiverse than a BART baseline. For FEVER [56] fact veriﬁcation, we achieve results within 
4.3% of\nstate-of-the-art pipeline models which use strong retrieval supervision. Finally, we demonstrate that\nthe
non-parametric memory can be replaced to update the models’ knowledge as the world changes.1\n2\nMethods\nWe 
explore RAG models, which use the input sequence x to r

RAG, or Retrieval-Augmented Generation, is a method that combines non-parametric and parametric memory to improve sequence-to-sequence tasks. The non-parametric memory is a dense vector index of Wikipedia, and a pre-trained neural retriever is used to access this information. The parametric memory is a pre-trained seq2seq transformer. These two components are combined in a probabilistic model trained end-to-end.

There are two variants of RAG: RAG-Sequence and RAG-Token. RAG-Sequence treats the retrieved document as a single latent variable that is marginalized to get the seq2seq probability p(y|x) via a top-K approximation. RAG-Token, on the other hand, allows the generator to choose content from several documents when producing an answer by drawing a different latent document for each target token and marginalizing accordingly.

The RAG models use a retriever pη(z|x) with parameters η that returns distributions over text passages given a query x and a generator pθ(yi|x, z, y1:i−1) pa

In [28]:
chatbot = gr.Chatbot(value = [[None, initial_msg]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
    demo.launch(debug=True, share=True, show_api=False)
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e

C:\Users\dbenn\AppData\Local\Temp\ipykernel_50348\1029931177.py:1: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(value = [[None, initial_msg]])
C:\Users\dbenn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\gradio\chat_interface.py:321: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://2cb647ab31ff8a9f58.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Retrieved Context for: What is BERT?

Prompt Sent to LLM: messages=[SystemMessage(content="\n    You are a knowledgeable research assistant who helps 
users understand technical AI papers. \n    \n    The user's question is: What is BERT?\n    \n    CONVERSATION 
HISTORY:\n    --- Conversation History ---\n\n\nUser previously responded with Tell me about RAG!\n\n\nAgent 
previously responded with RAG, or Retrieval-Augmented Generation, is a method that combines non-parametric and 
parametric memory to improve sequence-to-sequence tasks. The non-parametric memory is a dense vector index of 
Wikipedia, and a pre-trained neural retriever is used to access this information. The parametric memory is a 
pre-trained seq2seq transformer. These two components are combined in a probabilistic model trained 
end-to-end.\n\nThere are two variants of RAG: RAG-Sequence and RAG-Token. RAG-Sequence treats the retrieved 
document as a single latent variable that is marginalized to get the seq2seq probability p(y|x) via a top-K 
approximation. RAG-Token, on the other hand, allows the generator to choose content from several documents when 
producing an answer by drawing a different latent document for each target token and marginalizing 
accordingly.\n\nThe RAG models use a retriever pη(z|x) with parameters η that returns distributions over text 
passages given a query x and a generator pθ(yi|x, z, y1:i−1) parametrized by θ to produce the next output token. 
The top K documents are retrieved using the retriever, and then the generator produces a distribution for the next 
output token for each document, before marginalizing.\n\nCompared to retrieve-and-edit style approaches, RAG 
techniques place less emphasis on lightly editing a retrieved item and more on aggregating content from several 
pieces of retrieved content. RAG also learns latent retrieval and retrieves evidence documents rather than related 
training pairs.\n\nRAG has been shown to outperform previous extractive approaches for extractive tasks and to 
generate responses that are more factual, specific, and diverse in knowledge-intensive generation tasks. It also 
achieved results within 4.3% of state-of-the-art pipeline models for FEVER fact verification and demonstrated that 
the non-parametric memory can be replaced to update the models’ knowledge as the world changes.\n\nSource:\n\n> We 
build RAG models where the parametric memory is a pre-trained seq2seq transformer, and the non-parametric memory is
a dense vector index of Wikipedia, accessed with a pre-trained neural retriever. We combine these components in a 
probabilistic model trained end-to-end (Fig. 1). The retriever provides latent documents conditioned on the input, 
and the seq2seq model then conditions on these latent documents together with the input to generate the output. We 
marginalize the latent documents with a top-K approximation, either on a per-output basis (assuming the same 
document is responsible for all tokens) or a per-token basis (where different documents are responsible for 
different tokens).\n\n> RAG-Sequence treats the retrieved document as a single latent variable that is marginalized
to get the seq2seq probability p(y|x) via a top-K approximation. Concretely, the top K documents are retrieved 
using the retriever, and the generator produces the output sequence probability for each document, which are then 
marginalized.\n\n> In the RAG-Token model we can draw a different latent document for each target token and 
marginalize accordingly. This allows the generator to choose content from several documents when producing an 
answer.\n\n> RAG does have some similarities with retrieve-and-edit style approaches, but it does not emphasize 
lightly editing a retrieved item. Instead, RAG focuses on aggregating content from several pieces of retrieved 
content.\n\n> RAG has been shown to outperform previous extractive approaches for extractive tasks and to generate 
responses that are more factual, specific, and diverse in knowledge-intensi

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2cb647ab31ff8a9f58.gradio.live
Closing server running on port: 7860
